In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from train import LitTransformer
from data import TransformerDataset
import torch
import yaml
from data import get_tgt_str, build_vocab, id2token
import pandas as pd

In [ ]:

#data_path = "/home/takeru/AlphaSymbol/data/prfndim/d3-a5-c3-r5.csv"
data_path = "/home/takeru/AlphaSymbol/data/prfndim/d3-a2-c3-r3-status.csv"
df = pd.read_csv(data_path)
dataset = TransformerDataset(df)
src_max_len = dataset.src_max_len
tgt_max_len = dataset.tgt_max_len
src_vocab = dataset.src_vocab
tgt_vocab = dataset.tgt_vocab

#model_path = "/home/takeru/AlphaSymbol/models/version_13/checkpoints/epoch=299-step=117000.ckpt"
#model_path = "/home/takeru/AlphaSymbol/models/version_14/checkpoints/epoch=4-step=3895.ckpt"
model_path = "/home/takeru/AlphaSymbol/models/d3-a2-c3-r3-status/version_15/checkpoints/epoch=99-step=700.ckpt"
model = LitTransformer.load_from_checkpoint(model_path)
model.eval()

inputs = [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]
#outputs = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
#outputs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#inputs = [(0,), (0,)]
#outputs = [0, 0]
#outputs = [1, 1]
src_str = get_tgt_str(inputs, outputs)
src_tensor = torch.tensor([src_vocab[i] for i in src_str])
src_tensor = torch.cat([src_tensor, torch.tensor([src_vocab["<pad>"] for _ in range(src_max_len - src_tensor.shape[0])])])
src_tensor = src_tensor.unsqueeze(0)
src_tensor = src_tensor.to(model.device)

out_str = torch.tensor(tgt_vocab["<sos>"]).reshape((1, ))
out_str = out_str.unsqueeze(0)
out_str = out_str.to(model.device)


with torch.no_grad():
    for i in range(tgt_max_len):
        pred = model(src_tensor, out_str)
        next_c = pred[0, -1, :]
        max_values, max_indices = torch.max(next_c, axis=0)
        out_str = torch.cat((out_str, max_indices.reshape(1, 1)), axis=1)
        if max_indices == tgt_vocab["<eos>"]:
            break

print([id2token(tgt_vocab, id) for id in out_str[0]])